# 11章 基盤モデルとモデルカタログ ファインチューニングデータセット作成

必要なパッケージをimportします。

In [2]:
import json
import os
import pandas as pd

from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential
from datasets.load import load_dataset
from dotenv import load_dotenv

`.env`から環境変数をロードします。

In [3]:
load_dotenv()

https://github.com/yahoojapan/JGLUE/tree/main/datasets/jsts-v1.1

本ノートブックではJSTSデータセットのうちtrain-v1.1.jsonを使用します。

JSTSは、STS（意味テキスト類似性）データセットの日本語版です。STSは文のペアの意味的な類似性を推定するタスクです。JSTSおよび以下で説明するJNLIの文は日本語版のMS COCOキャプションデータセットおよびYJキャプションデータセット（Miyazaki and Shimizu, 2016）から抽出されています。

In [4]:
file_path = 'train-v1.1.json'
df = pd.read_json(file_path, lines=True)

df.head()

システムプロンプトを定義します。

In [5]:
system_message = {
    "role": "system",
    "content": "あなたはテキストの類似性を評価することができる熟練の日本語話者です。2つの日本語の文章を入力として受け入れ、"
                "0 から 5 までの間の数値 (小数第1位までの値) で類似度を評価します。文章の類似性が高い場合に大きい数値をとり、"
                "類似性が低い場合は小さい値をとります。例えば同じ文章や完全に意味が一致している場合は5となります。"
                "入力形式は文章1 : <文章1>、文章2 : <文章2> です。"
                "出力はスコアの値のみです。"
                "あなたの努力は素晴らしい成果をもたらすでしょう。頑張ってタスクをやり遂げてください。"
}

データセットを構築します。データセットは、システムプロンプトの次にユーザーからの入力として2つの文が与えられ、アシスタントの応答としてその類似度のみを返却するという形式です。

今回は時間短縮のためデータセット件数を100件に制約しています。

整形したデータセットは、1行が1つのJSONで構成されるjsonl形式で保存します。

In [6]:
output_data = []
output_file_name = 'output.jsonl'

for index, row in df.iterrows():
    user_message = {
        "role": "user",
        "content": f"文章 1 : {row['sentence1']}, 文章 2 : {row['sentence2']}"
    }
    
    assistant_message = {
        "role": "assistant",
        "content": str(row['label'])
    }
    
    output_data.append({
        "messages": [system_message, user_message, assistant_message]
    })
    
    if index == 99:
        break
    
with open(output_file_name, 'w', encoding='utf-8') as f:
    for entry in output_data:
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')

Azure Machine Learning上で行うファインチューニングではHugging Faceのdatasetsパッケージが使用されています。以下関数の実行が通ればファインチューニングジョブでも問題なくデータセットを読み込むことができます。

In [7]:
ds = load_dataset('json', data_files=f"./{output_file_name}",split='train', **{})

Azure Machine Learning上にデータセットをDataアセットとして登録する前準備として、Azure Machine Learningに接続する`MLClient`インスタンスを初期化します。

In [8]:
subscription_id = os.environ.get("SUBSCRIPTION_ID")
resource_group = os.environ.get("RESOURCE_GROUP")
workspace = os.environ.get("AML_WORKSPACE_NAME")

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

Azure Machine Learning上にデータセットをDataアセットとして登録します。

In [9]:
finetuning_data = Data(
    path=output_file_name,
    type=AssetTypes.URI_FILE,
    description="JSTSデータセットを利用したfine-tuningデータセット",
    name="semantic-text-similarity-fine-tune",
)

ml_client.data.create_or_update(finetuning_data)